# Machine Learning

In this file, instructions how to approach the challenge can be found.

We are going to work on different types of Machine Learning problems:

- **Regression Problem**: The goal is to predict delay of flights.
- **(Stretch) Multiclass Classification**: If the plane was delayed, we will predict what type of delay it is (will be).
- **(Stretch) Binary Classification**: The goal is to predict if the flight will be cancelled.

## Main Task: Regression Problem

The target variable is **ARR_DELAY**. We need to be careful which columns to use and which don't. For example, DEP_DELAY is going to be the perfect predictor, but we can't use it because in real-life scenario, we want to predict the delay before the flight takes of --> We can use average delay from earlier days but not the one from the actual flight we predict.  

For example, variables **CARRIER_DELAY, WEATHER_DELAY, NAS_DELAY, SECURITY_DELAY, LATE_AIRCRAFT_DELAY** shouldn't be used directly as predictors as well. However, we can create various transformations from earlier values.

We will be evaluating your models by predicting the ARR_DELAY for all flights **1 week in advance**.

In [24]:
# Base Functions
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import datasets
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import plotly.express as px
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
import seaborn as sns

In [25]:
#data = pd.read_csv('../data/training_set/TRAIN_ME.csv')
dataset = pd.read_csv('../data/training_set/TRAIN_ME_carriers.csv')

In [26]:
data = dataset.copy()

#### Regression Section

In [27]:
dfm = data.copy()
dfm.columns

Index(['Unnamed: 0', 'origin_airport_id', 'dest_airport_id', 'day', 'month',
       'year', 'crs_dep_time', 'crs_elapsed_time', 'distance', 'month_delay',
       'carrier_delay', 'carrier_perc_delay', 'departures_performed', 'seats',
       'passengers', 'percent_filled', 'arr_delay', 'windspeeds',
       'weather_desc', 'visibilities', 'precipitations', 'mkt_carrier'],
      dtype='object')

In [28]:
dfm = pd.get_dummies(dfm, columns=['mkt_carrier'], prefix='mkt_carrier', drop_first=True)
dfm = pd.get_dummies(dfm, columns=['dest_airport_id'], prefix='DEST', drop_first=True)
dfm = pd.get_dummies(dfm, columns=['crs_dep_time'], prefix='CRS_DEP_TIME', drop_first=True)
dfm = pd.get_dummies(dfm, columns=['weather_desc'], prefix='weather', drop_first=True) 
dfm = pd.get_dummies(dfm, columns=['month'], prefix='month', drop_first=True)
dfm = pd.get_dummies(dfm, columns=['day'], prefix='day', drop_first=True)

In [33]:
dfm = dfm.drop(['Unnamed: 0'],axis=1)

In [1]:
len(dfm.columns)

NameError: name 'dfm' is not defined

In [34]:
dfm.to_csv('../data/training_set/xgb_reg_data.csv')

In [14]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
data['mkt_carrier'] = le.fit_transform(data['mkt_carrier'])

In [5]:
data = data.drop(['Unnamed: 0'],axis=1)


In [6]:
data['flight_status'] = (data['arr_delay'] > 15).replace([True,False],[1,0])

In [7]:
data.drop(['arr_delay'],axis=1,inplace=True)

In [8]:
data['speed']=data['distance']/data['crs_elapsed_time']

In [10]:
len(data.columns)

22

In [12]:
hot_set = data.copy()

In [28]:
sample1 = data.loc[data['flight_status'] == 1]

In [21]:
sample2 = data.loc[data['flight_status'] == 0]

In [22]:
sample_2 = sample2.sample(n=757903)

In [23]:
sample = sample1.append(sample_2)

In [29]:
sample = sample.sample(n=200000)


In [36]:
X = dfm.drop(['arr_delay'], axis=1)
y = dfm['arr_delay']



In [26]:
y.value_counts()

0    100108
1     99892
Name: flight_status, dtype: int64

### PCA

In [1]:

# from sklearn.datasets import load_diabetes

# pca = PCA()
# pca.fit(sample)
# exp_var_cumul = np.cumsum(pca.explained_variance_ratio_)

# px.area(
#     x=range(1, exp_var_cumul.shape[0] + 1),
#     y=exp_var_cumul,
#     labels={"x": "# Components", "y": "Explained Variance"}
# )

In [ ]:
# PCA with 4 components
plt.style.use('ggplot')

scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)
# The PCA model
pca = PCA(n_components=4) # estimate only 4 PCs
X_new = pca.fit_transform(X) # project the original data into the PCA space

In [79]:
X_new[0]

array([-2945.5704069 , -1424.83175664,  -235.55666193,   473.75188373])

In [14]:
#### Optional Plot to see pca vs x

# fig, axes = plt.subplots(1,2)
# axes[0].scatter(X[:,0], X[:,1], c=y)
# axes[0].set_xlabel('x1')
# axes[0].set_ylabel('x2')
# axes[0].set_title('Before PCA')
# axes[1].scatter(X_new[:,0], X_new[:,1], c=y)
# axes[1].set_xlabel('PC1')
# axes[1].set_ylabel('PC2')
# axes[1].set_title('After PCA')
# plt.show()

In [105]:
# Put this into model
X_pca = pd.DataFrame(X_new)

In [106]:
X_pca

,0,1,2,3
0,-0.778713,-0.267451,-0.857667,0.006228
1,1.281522,0.089195,2.702915,0.864878
2,-0.502834,0.809387,2.860419,0.238574
3,2.806432,-2.287953,1.708544,0.482690
4,2.619972,0.562542,-1.607023,-0.560244
...,...,...,...,...
999995,-3.172378,-0.003962,-1.187154,-0.485970
999996,-1.417283,0.682981,-1.012900,-0.114353
999997,-1.490590,-0.884019,1.885885,0.836634
999998,-1.103703,-0.189567,-1.827589,0.479622


In [27]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [28]:
# For classification
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)






### Regression

In [19]:
# Naive Bayes

from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, y_train)

from sklearn.metrics import confusion_matrix, accuracy_score
y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[627782      0]
 [151886      0]]


0.8051914404592724

## Classification

### KNN

In [119]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
classifier.fit(X_train, y_train)

KNeighborsClassifier()

In [120]:
from sklearn.metrics import confusion_matrix, accuracy_score
y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

KeyboardInterrupt: 

### Logistic Regression

In [41]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0)
classifier.fit(X_train, y_train)

LogisticRegression(random_state=0)

In [42]:
from sklearn.metrics import confusion_matrix, accuracy_score
y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[51476 48663]
 [48512 51349]]


0.514125

### SVM

In [ ]:
from sklearn.svm import SVC
classifier = SVC(kernel = 'linear', random_state = 0)
classifier.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

### Kernel SVM

In [ ]:
from sklearn.svm import SVC
classifier = SVC(kernel = 'sigmoid', random_state = 0)
classifier.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

### Random Forest Classification

In [98]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 10, criterion = 'gini', random_state = 0)
classifier.fit(X_train, y_train)

RandomForestClassifier(n_estimators=10, random_state=0)

In [109]:
from sklearn.metrics import confusion_matrix, accuracy_score
y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[66392 33555]
 [45818 54235]]


0.603135

### Ensemble Methods

In [34]:
!pip install xgboost

     |████████████████████████████████| 1.2 MB 3.3 MB/s eta 0:00:01


In [29]:
import xgboost as xgb

In [30]:
xg_reg = xgb.XGBClassifier()

In [ ]:
xg_reg.fit(X_train,y_train)

preds = xg_reg.predict(X_test)

### Feature Engineering

Feature engineering will play a crucial role in this problems. We have only very little attributes so we need to create some features that will have some predictive power.

- weather: we can use some weather API to look for the weather in time of the scheduled departure and scheduled arrival.
- statistics (avg, mean, median, std, min, max...): we can take a look at previous delays and compute descriptive statistics
- airports encoding: we need to think about what to do with the airports and other categorical variables
- time of the day: the delay probably depends on the airport traffic which varies during the day.
- airport traffic
- unsupervised learning as feature engineering?
- **what are the additional options?**: Think about what we could do more to improve the model.

### Feature Selection / Dimensionality Reduction

We need to apply different selection techniques to find out which one will be the best for our problems.

- Original Features vs. PCA conponents?

In [2]:
from sklearn.datasets import load_digits
from sklearn.feature_selection import SelectKBest, chi2, SelectPercentile
X, y = load_digits(return_X_y=True)
X_new = SelectKBest('SelectPercentile', k=10).fit_transform(X, y)
X_new.shape

TypeError: The score function should be a callable, SelectPercentile (<class 'str'>) was passed.

### Modeling

Use different ML techniques to predict each problem.

- linear / logistic / multinomial logistic regression
- Naive Bayes
- Random Forest
- SVM
- XGBoost
- The ensemble of your own choice

### Evaluation

You have data from 2018 and 2019 to develop models. Use different evaluation metrics for each problem and compare the performance of different models.

You are required to predict delays on **out of sample** data from **first 7 days (1st-7th) of January 2020** and to share the file with LighthouseLabs. Sample submission can be found in the file **_sample_submission.csv_**

======================================================================
## Stretch Tasks

### Multiclass Classification

The target variables are **CARRIER_DELAY, WEATHER_DELAY, NAS_DELAY, SECURITY_DELAY, LATE_AIRCRAFT_DELAY**. We need to do additional transformations because these variables are not binary but continuos. For each flight that was delayed, we need to have one of these variables as 1 and others 0.

It can happen that we have two types of delays with more than 0 minutes. In this case, take the bigger one as 1 and others as 0.

### Binary Classification

The target variable is **CANCELLED**. The main problem here is going to be huge class imbalance. We have only very little cancelled flights with comparison to all flights. It is important to do the right sampling before training and to choose correct evaluation metrics.